In [1]:
# import librairies
import requests
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np


In [2]:
# create the list movie_cards, with as many nested lists as pages, with all page entity cards inside.
pages = 25
url_param = 'http://www.allocine.fr'
fill_val = ''
movies_cards = []

for page in range(pages):
    url = f'http://www.allocine.fr/dvd/meilleurs/?page={page}'
    r = requests.get(url)
    allocine = BeautifulSoup(r.text, 'lxml')
    cards = allocine.find_all('div', class_='entity-card-list')
    movies_cards.append(cards)


In [3]:
# fill our movie lists
movie_title = []
movie_url = []
movie_acteurs = []
movie_resume = []
movie_presse = []
movie_spectateurs = []
movie_amis = []

for pages in movies_cards:
    for card in pages:
        # add movies title
        movie_title.append(card.find('a', class_='meta-title-link', href=True).text)

        # add movies url
        movie_url.append(url_param + card.find('a', class_='meta-title-link', href=True)['href'])
        
        # add movies actors
        if 'meta-body-actor' in str(card):
            actors_blocs = card.find('div', class_='meta-body-actor').text.splitlines()
            actors_list = []
            
            for list_elements in actors_blocs:
                m = list_elements.strip()
                if m != '' and m != 'Avec':
                    actors_list.append(m)
            movie_acteurs.append(' '.join(actors_list))
            
        else:
            movie_acteurs.append(fill_val)
        
        # add movies resume
        if 'content-txt' in str(card):
            movie_resume.append(card.find('div', class_='content-txt').text.strip())
        else:
            movie_resume.append(fill_val)
        
        # add press, spectators and friends movie ratings
        if 'rating-item' in str(card):
            rating = card.find_all('div', class_='rating-item')
            i = 0
            tour = 1
            for elmt in rating:
                # we have to analyse each blocs separatly (cause same span/div class names)
                if tour%3 == 1:
                    if 'Presse' in str(rating[i]):
                        u = elmt.find_all('span', class_="stareval-note")[0].text
                        movie_presse.append(u)
                    else:
                        movie_presse.append(fill_val)
                        tour += 1
           
                if tour%3 == 2:
                    if 'Spectateur' in str(rating[i]):
                        v = elmt.find_all('span', class_="stareval-note")[0].text
                        movie_spectateurs.append(v)
                    else: 
                        movie_spectateurs.append(fill_val)
            
                if tour%3 == 0:
                    if 'Amis' in str(rating[i]):
                        v = elmt.find_all('span', class_="stareval-note")[0].text
                        movie_amis.append(v)
                    else:
                        movie_amis.append(fill_val)
            
                i += 1
                tour += 1
        else:
            movie_presse.append(fill_val)
            movie_spectateurs.append(fill_val)
            movie_amis.append(fill_val)

In [4]:
# check length of lists
print(len(movie_title))
print(len(movie_url))
print(len(movie_acteurs))
print(len(movie_resume))
print(len(movie_presse))
print(len(movie_spectateurs))
print(len(movie_amis))

375
375
375
375
375
375
375


In [5]:
# create our dataframe
allocine_recent_movies = pd.DataFrame(
    {'Title': movie_title,
     'Press Rating' : movie_presse,
     'Spectator Rating' : movie_spectateurs,
     'Actors': movie_acteurs,
     'Resume' : movie_resume,
     'Allociné URL' : movie_url
    })

In [6]:
# set full display on texts
pd.set_option('display.max_colwidth', -1)

In [7]:
# let's take a look
allocine_recent_movies.head()

,Title,Press Rating,Spectator Rating,Actors,Resume,Allociné URL
0,Green Book : Sur les routes du sud,"4,0","4,5","Viggo Mortensen, Mahershala Ali, Linda Cardellini","En 1962, alors que règne la ségrégation, Tony Lip, un videur italo-américain du Bronx, est engagé pour conduire et protéger le Dr Don Shirley, un pianiste noir de renommée mondiale, lors d’une tournée de concerts.",http://www.allocine.fr/film/fichefilm_gen_cfilm=256661.html
1,Bohemian Rhapsody,"3,0","4,5","Rami Malek, Gwilym Lee, Lucy Boynton","Du succès fulgurant de Freddie Mercury à ses excès, risquant la quasi-implosion du groupe, jusqu’à son retour triomphal sur scène lors du concert Live Aid, alors qu’il était frappé par la maladie, découvrez la vie exceptionnelle d’un homme qui continue d’inspirer les outsiders, les rêveurs et tous ceux qui aiment la musique.",http://www.allocine.fr/film/fichefilm_gen_cfilm=185719.html
2,Capharnaüm,"3,0","4,4","Zain Al Rafeea, Cedra Izam, Nadine Labaki","À l'intérieur d'un tribunal, Zain, un garçon de 12 ans, est présenté devant le juge. À la question : "" Pourquoi attaquez-vous vos parents en justice ? "", Zain lui répond : "" Pour m'avoir donné la vie ! "". Capharnaüm retrace l'incroyable parcours de cet enfant en quête d'identité et qui se rebelle contre la vie qu'on cherche à lui imposer.",http://www.allocine.fr/film/fichefilm_gen_cfilm=251090.html
3,Edmond,"3,6","4,4","Thomas Solivérès, Olivier Gourmet, Mathilde Seigner","Décembre 1897, Paris. Edmond Rostand n’a pas encore trente ans mais déjà deux enfants et beaucoup d’angoisses. Il n’a rien écrit depuis deux ans. En désespoir de cause, il propose au grand Constant Coquelin une pièce nouvelle, une comédie héroïque, en vers, pour les fêtes.",http://www.allocine.fr/film/fichefilm_gen_cfilm=256968.html
4,Spider-Man : New Generation,"4,2","4,4","Stéphane Bak, Camélia Jordana, Olivier Giroud","SPIDER-MAN : NEW GENERATION présente Miles Morales, un adolescent vivant à Brooklyn, et révèle les possibilités illimitées du Spider-Verse, un univers où plus d’un peut porter le masque…",http://www.allocine.fr/film/fichefilm_gen_cfilm=233998.html


In [8]:
# dataframe to HTML
allocine_recent_movies.to_html('.../190814_allocine_recent_movies.html')

In [9]:
# dataframe to CSV
allocine_recent_movies.to_csv('.../190814_allocine_recent_movies.csv')

In [ ]:
# dataframe to PDF
# tries with pdfkit, and PyQT4, but no success.